In [86]:
import numpy as np
from glob import glob
import pandas as pd
import random
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics import auc
import pickle
import pyod
from pyod.utils.stat_models import pairwise_distances_no_broadcast
import warnings
from icecream import ic
from src.constant import translation_dict
from torch.utils.data import random_split, DataLoader
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_auc_score, precision_score, recall_score, confusion_matrix

warnings.filterwarnings("ignore")


In [168]:
import pandas as pd
import glob

# 获取所有的CSV文件
csv_path = 'data/Train'
pkl_path = 'data/pkl/Train'
csv_files = glob.glob(csv_path+'/*.csv')
pkl_files = glob.glob(pkl_path+'/*.pkl')

In [169]:
# with_label_df_list = [pd.read_csv(f) for f in csv_files]
# non_label_df_list = [pd.read_csv(f).drop('label', axis=1)  for f in csv_files]


In [177]:
df_list = [pd.read_csv(f).replace({'label': {10: 1}}) for f in csv_files]


# random.shuffle(df_list)

train_percent=0.7
train_size = int(len(df_list) * train_percent)
test_size = len(df_list) - train_size

# X_list = [df.drop('label', axis=1) for df in df_list]
# y_list = [df['label'] for df in df_list]

train_dataset, test_dataset = random_split(dataset=df_list, lengths=[train_size, test_size])
# train_labels, test_labels = random_split(dataset=y_list, lengths=[train_size, test_size])

# train_dataset = df_list[:train_size]
# test_dataset = df_list[train_size:]
ic(len(train_dataset), len(test_dataset));

ic| len(train_dataset): 19872, len(test_dataset): 8517


In [178]:
test_dataset[3]

,volt,current,soc,max_single_volt,min_single_volt,max_temp,min_temp,timestamp,label
0,165.9,-4.3,46.4,1.845,1.841,114.0,96.0,40014.0,0
1,165.9,-4.3,46.4,1.845,1.842,114.0,96.0,40015.0,0
2,165.9,-4.3,46.4,1.845,1.842,114.0,96.0,40016.0,0
3,165.9,-4.2,46.4,1.845,1.842,114.0,96.0,40017.0,0
4,165.9,-4.3,46.4,1.845,1.842,114.0,96.0,40018.0,0
...,...,...,...,...,...,...,...,...,...
251,166.3,-4.3,47.0,1.850,1.846,114.0,96.0,40710.0,0
252,166.3,-4.3,47.0,1.850,1.847,114.0,96.0,40720.0,0
253,166.3,-4.3,47.0,1.851,1.846,114.0,96.0,40730.0,0
254,166.3,-4.2,47.0,1.850,1.847,114.0,96.0,40740.0,0


In [179]:
ic(train_dataset[0].info(), test_dataset[0].info());

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   volt             256 non-null    float64
 1   current          256 non-null    float64
 2   soc              256 non-null    float64
 3   max_single_volt  256 non-null    float64
 4   min_single_volt  256 non-null    float64
 5   max_temp         256 non-null    float64
 6   min_temp         256 non-null    float64
 7   timestamp        256 non-null    float64
 8   label            256 non-null    int64  
dtypes: float64(8), int64(1)
memory usage: 18.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   volt             256 non-null    float64
 1   current          256 non-null    float64
 2   soc              256 non-null    float64
 3   

ic| train_dataset[0].info(): None, test_dataset[0].info(): None


In [180]:
class MyDataset(Dataset):
    def __init__(self, dataset):
        self.data = dataset
        self.length=len(self.data)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        X = self.data[idx].drop(['label'], axis=1).astype('float32').values
        y = self.data[idx]['label'].astype('float32').values[0]
        
        # 使用 reshape 方法将 y 的形状改为 [1,]
        y = np.reshape(y, (1,))

        return X, y


batch_size = 64
train_data = MyDataset(dataset = train_dataset)
test_data = MyDataset(dataset = test_dataset)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True,drop_last=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True,drop_last=False)


In [203]:
# 迭代 DataLoader
for batch_idx, (data, target) in enumerate(train_loader):
    print("Batch idx {}, data shape {}, target shape {}".format(
        batch_idx, data.shape, target.shape))
    
    ic(data.shape, target.shape);
    ic(data.dtype, target.dtype);
    # ic(data)
    ic(target.sum());
    # 你可以在这里添加你的测试代码，例如检查数据和目标的形状、类型等
    # assert data.shape == (64, 1, 28, 28)
    # assert target.shape == (64,)
    
    break;

Batch idx 0, data shape torch.Size([64, 256, 8]), target shape torch.Size([64, 1])


ic| data.shape: torch.Size([64, 256, 8])
    target.shape: torch.Size([64, 1])
ic| data.dtype: torch.float32, target.dtype: torch.float32
ic| target.sum(): tensor(6.)


# 模型 MLP

In [204]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(in_features=8*256, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=1)
        self.m = nn.Sigmoid()

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.m(x)
        return x

model = MyNet()


In [205]:
max_epoch = 10
now_step=10
lr = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epoch, eta_min=lr)

model.train()

criterion = nn.BCELoss()

for epoch in range(max_epoch):
    for step, (data, label) in enumerate(iterable=train_loader):
        # ic(data.shape, label.shape)
        # break
        now_step += 1
        predicted = model(data)
        # ic(label.shape, predicted.shape)
        loss = criterion(predicted, label)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if now_step % 100 == 0:
            print("epoch: {}, batch: {}, loss is: {}".format(epoch, step, loss.item()))

epoch: 0, batch: 89, loss is: 14.0625
epoch: 0, batch: 189, loss is: 18.75
epoch: 0, batch: 289, loss is: 14.0625
epoch: 1, batch: 78, loss is: 20.3125
epoch: 1, batch: 178, loss is: 18.75
epoch: 1, batch: 278, loss is: 18.75
epoch: 2, batch: 67, loss is: 17.1875
epoch: 2, batch: 167, loss is: 20.3125
epoch: 2, batch: 267, loss is: 25.0
epoch: 3, batch: 56, loss is: 29.6875
epoch: 3, batch: 156, loss is: 32.8125
epoch: 3, batch: 256, loss is: 17.1875
epoch: 4, batch: 45, loss is: 14.0625
epoch: 4, batch: 145, loss is: 20.3125
epoch: 4, batch: 245, loss is: 17.1875
epoch: 5, batch: 34, loss is: 17.1875
epoch: 5, batch: 134, loss is: 18.75
epoch: 5, batch: 234, loss is: 15.625
epoch: 6, batch: 23, loss is: 18.75
epoch: 6, batch: 123, loss is: 15.625
epoch: 6, batch: 223, loss is: 18.75
epoch: 7, batch: 12, loss is: 12.5
epoch: 7, batch: 112, loss is: 18.75
epoch: 7, batch: 212, loss is: 20.3125
epoch: 8, batch: 1, loss is: 15.625
epoch: 8, batch: 101, loss is: 7.8125
epoch: 8, batch: 201

# 预测

In [210]:
model.eval()

correct = 0
total = 0
# 测试模型
label_sum = 0
with torch.no_grad():
    for idx, (data, label) in enumerate(iterable=test_loader):
        outputs = model(data)
        ic(outputs)
        ic(label)
        _, predicted = torch.max(input=outputs.data, dim=1)
        label = torch.argmax(label, dim=1)
        ic(label.shape, predicted.shape)
        break    
        # ic(label.shape, predicted.shape)
        total += label.size(dim=0)
        correct += (predicted == label).sum().item()
        # break

# print(f'正确率: {correct / total}')

ic| outputs: tensor([[0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
                     [0.],
 

In [ ]:
train_dataset[1].info()